# import

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from random import randint,sample,random as rnd
import networkx as nx
from tqdm.autonotebook import tqdm
from scipy.stats import linregress
from time import time as timer

In [2]:
from networklib import *

In [3]:
def common_neiz(neiz,i=None,j=None):
    if j is None:
        if i is None:
            n = len(neiz)
            neiz_set = [set(nei) for nei in neiz]
            J = np.zeros((n,n),int)
            for a in range(1,n):
                J[a,:a] = [len(neiz_set[a].intersection(neiz_set[b])) for b in range(a)]
            return J + J.T
        else:
            n = len(neiz)
            neiz_set = [set(nei) for nei in neiz]
            return [len(neiz_set[j].intersection(neiz_set[i])) for i in range(n)]

    else:
        if j is None:
            n = len(neiz)
            neiz_set = [set(nei) for nei in neiz]
            return [len(neiz_set[i].intersection(neiz_set[j])) for j in range(n)]
        else:
            return set(neiz[i]).intersection(neiz[j])

In [4]:
def func_Xij(neiz,degree,adj=None,return_J=False):
    n = len(neiz)
    if adj is None:
        adj = np.zeros((n,n),bool)
        for i in range(n):
            adj[i,neiz[i]] = True
    J = common_neiz(neiz)
    X = np.zeros((n,n))
    for i in range(n):
        minz = degree.copy()
        minz[degree>degree[i]] = degree[i]
        X[i] = (J[i]+adj[i])/(minz+1-adj[i])
    return X,J if return_J else X

In [7]:
nodes1 = [i for i in 'abcdefghijk']
nodes1

['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k']

In [20]:
dict1 = {i:j for j,i in enumerate(nodes1)}

In [26]:
neiz = [
    ['b','c'],
    ['c'],
    ['d'],
    ['e','i'],
    ['f','g'],
    ['g'],
    ['h'],
    ['i','j'],
    ['j','k'],
    ['k'],
    [],
]

In [23]:
neiz2

[['b', 'c', 'b', 'c'],
 ['c', 'a', 'c'],
 ['d', 'a', 'b', 'd'],
 ['e', 'i', 'c', 'e'],
 ['f', 'g', 'd', 'f', 'g'],
 ['g', 'e', 'g'],
 ['h', 'e', 'f', 'h'],
 ['i', 'j', 'g'],
 ['j', 'k', 'd', 'h'],
 ['k', 'h', 'i']]

In [27]:
neiz2 = neiz.copy()
for i,node in enumerate(nodes1[:-1]):
    # neiz2.append(neiz[i])
    for j in neiz[i]:
        neiz2[dict1[j]].append(node)
neiz2

[['b', 'c', 'b', 'c'],
 ['c', 'a', 'c'],
 ['d', 'a', 'b', 'd'],
 ['e', 'i', 'c', 'e', 'i'],
 ['f', 'g', 'd', 'f', 'g'],
 ['g', 'e', 'g'],
 ['h', 'e', 'f', 'h'],
 ['i', 'j', 'g', 'i', 'j'],
 ['j', 'k', 'd', 'h', 'j'],
 ['k', 'h', 'i'],
 ['i', 'j']]

In [36]:
sorted(neiz[0])

['b', 'c']

In [37]:
neiz = [sorted(list(set(neiz2[i]))) for i in range(len(nodes1))]

In [38]:
for i,node in enumerate(nodes1):
    print(node)
    print(neiz[i])
    print("")

a
['b', 'c']

b
['a', 'c']

c
['a', 'b', 'd']

d
['c', 'e', 'i']

e
['d', 'f', 'g']

f
['e', 'g']

g
['e', 'f', 'h']

h
['g', 'i', 'j']

i
['d', 'h', 'j', 'k']

j
['h', 'i', 'k']

k
['i', 'j']



In [40]:
dict1

{'a': 0,
 'b': 1,
 'c': 2,
 'd': 3,
 'e': 4,
 'f': 5,
 'g': 6,
 'h': 7,
 'i': 8,
 'j': 9,
 'k': 10}

In [44]:
[dict1[j] for j in neiz]
# neiz

[['b', 'c'],
 ['a', 'c'],
 ['a', 'b', 'd'],
 ['c', 'e', 'i'],
 ['d', 'f', 'g'],
 ['e', 'g'],
 ['e', 'f', 'h'],
 ['g', 'i', 'j'],
 ['d', 'h', 'j', 'k'],
 ['h', 'i', 'k'],
 ['i', 'j']]

In [45]:
nei_n = []
for i in range(len(nodes1)):
    nei_n.append([dict1[j] for j in neiz[i]])
nei_n

[[1, 2],
 [0, 2],
 [0, 1, 3],
 [2, 4, 8],
 [3, 5, 6],
 [4, 6],
 [4, 5, 7],
 [6, 8, 9],
 [3, 7, 9, 10],
 [7, 8, 10],
 [8, 9]]

In [54]:
degree = np.array([len(nei) for nei in neiz])

In [60]:
X,J = func_Xij(nei_n,degree)

## Girvan-Newman (GN) Benchmark

In [13]:
def GirvanNewmanBenchmark(p_int=0.5,p_ext=0.1):
    n = 128
    C = 4
    nc = n//C
    nodes = np.arange(n).reshape(C,-1)
    neiz = [[] for _ in range(n)]
    for c in range(C):
        for ii,i in enumerate(nodes[c,:-1]):
            to_link = np.random.random(nc-ii-1) < p_int
            jz = list(nodes[c,ii+1:][to_link])
            neiz[i].extend(jz)
            for j in jz:
                neiz[j].append(i)
            for c2 in range(c+1,C):
                to_link = np.random.random(nc) < p_ext
                jz = list(nodes[c2][to_link])
                neiz[i].extend(jz)
                for j in jz:
                    neiz[j].append(i)
    return neiz,np.array([len(nei) for nei in neiz])

In [7]:
neiz , degree = GirvanNewmanBenchmark()

In [54]:
neiz,degree = randomNet_test_connceted(200,0.1,1,0,0,0)

In [71]:
def common_neiz(neiz,i=None,j=None):
    if j is None:
        if i is None:
            n = len(neiz)
            neiz_set = [set(nei) for nei in neiz]
            J = np.zeros((n,n),int)
            for a in range(1,n):
                J[a,:a] = [len(neiz_set[a].intersection(neiz_set[b])) for b in range(a)]
            return J + J.T
        else:
            n = len(neiz)
            neiz_set = [set(nei) for nei in neiz]
            return [len(neiz_set[j].intersection(neiz_set[i])) for i in range(n)]

    else:
        if j is None:
            n = len(neiz)
            neiz_set = [set(nei) for nei in neiz]
            return [len(neiz_set[i].intersection(neiz_set[j])) for j in range(n)]
        else:
            return set(neiz[i]).intersection(neiz[j])

In [94]:
def func_Xij(neiz,degree,adj=None,return_J=False):
    n = len(neiz)
    if adj is None:
        adj = np.zeros((n,n),bool)
        for i in range(n):
            adj[i,neiz[i]] = True
    J = common_neiz(neiz)
    X = np.zeros((n,n))
    for i in range(n):
        minz = degree.copy()
        minz[degree>degree[i]] = degree[i]
        X[i] = J[i]/(minz+1-adj[i])
    return X,J if return_J else X

In [96]:
J = common_neiz(neiz)
X = func_Xij(neiz,degree)

In [90]:
i,j = np.random.randint(0,200,2)
i,j

(69, 81)

In [97]:
adj[i,j]

False

In [72]:
J = common_neiz(neiz)